Import Libraries

In [1]:
import tensorflow as tf
import tensorflow.experimental.numpy as tnp
import matplotlib.pyplot as plt
from tensorflow import keras
from __future__ import print_function
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPool2D
from keras import backend as k

Variables:
batch: the process of splitting the training dataset in n batches (mini-batches)
classes: number of classifications (labels) of the data
epochs: variations, one epoch is one forward pass + one backward pass on training

In [2]:
batch_size = 128
num_classes = 10
epochs = 12

Assign training and test data

In [3]:
img_rows, img_cols = 28,28
(x_train,y_train),(x_test,y_test) =   mnist.load_data()

Reshape the images

In [4]:
if k.image_data_format()=='channels_first':
    x_train=x_train.reshape(x_train.shape[0],img_rows,img_cols,1)
    x_test=x_test.reshape(x_test.shape[0],img_rows,img_cols,1)
else:
    x_train=x_train.reshape(x_train.shape[0],img_rows,img_cols,1)
    x_test=x_test.reshape(x_test.shape[0],img_rows,img_cols,1)

input_shape=(img_rows,img_cols,1)
x_train = x_train/255.0
x_test=x_test/255.0
print('x_train shape:',x_train.shape,'\nx_test shape:',x_test.shape)

x_train shape: (60000, 28, 28, 1) 
x_test shape: (10000, 28, 28, 1)


Convert class vectors to binary class matrices

In [5]:
y_train=keras.utils.to_categorical(y_train,num_classes)
y_test=keras.utils.to_categorical(y_test,num_classes)

Design the CNN architecture

In [6]:
model=Sequential()

model.add( Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=input_shape) )
model.add( Conv2D(64,kernel_size=(3,3),activation='relu') )
model.add( MaxPool2D(pool_size=(2,2)) )
model.add( Conv2D(64,kernel_size=(3,3),activation='relu') )
model.add( Flatten() )
model.add( Dense(128,activation='relu') )
model.add( Dense(num_classes,activation='softmax') )
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 64)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 6400)              0         
                                                                 
 dense (Dense)               (None, 128)               819328    
                                                        

Compile the model

In [7]:
model.compile(optimizer=keras.optimizers.Adadelta(),
              loss=keras.losses.categorical_crossentropy,
              metrics=['accuracy']
             )
model.fit(x_train,y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test,y_test)
          )

Epoch 1/12
469/469 [==============================] - 17s 16ms/step - loss: 2.2674 - accuracy: 0.2083 - val_loss: 2.2258 - val_accuracy: 0.2892
Epoch 2/12
469/469 [==============================] - 9s 18ms/step - loss: 2.1667 - accuracy: 0.3962 - val_loss: 2.0844 - val_accuracy: 0.5052
Epoch 3/12
469/469 [==============================] - 7s 15ms/step - loss: 1.9484 - accuracy: 0.6091 - val_loss: 1.7466 - val_accuracy: 0.7109
Epoch 4/12
469/469 [==============================] - 6s 13ms/step - loss: 1.4686 - accuracy: 0.7439 - val_loss: 1.1354 - val_accuracy: 0.7859
Epoch 5/12
469/469 [==============================] - 8s 17ms/step - loss: 0.9371 - accuracy: 0.7896 - val_loss: 0.7414 - val_accuracy: 0.8183
Epoch 6/12
469/469 [==============================] - 7s 15ms/step - loss: 0.6817 - accuracy: 0.8188 - val_loss: 0.5850 - val_accuracy: 0.8409
Epoch 7/12
469/469 [==============================] - 7s 15ms/step - loss: 0.5710 - accuracy: 0.8388 - val_loss: 0.5097 - val_accuracy: 0.859

Evaluate the model on test data

In [8]:
score=model.evaluate(x_test,y_test,verbose=0)
print('\nTest loss=',score[0],'\nTest Accuracy=',score[1]*100,'%')


Test loss= 0.3782796561717987 
Test Accuracy= 89.42000269889832 %


Save the Model

In [9]:
model.save('simpleCNNmodel.h5')
print("model is saved")

model is saved
